In [1]:
import os
import json
import ast
import math
import pandas as pd
import gzip
import pickle as pkl
from collections import defaultdict
from datetime import datetime

In [2]:
os.getcwd()

'e:\\Workspaces\\SASRec.pytorch\\notebooks'

In [ ]:
ROOT_DIR = os.path.join(os.getcwd(), "..", "data", "Steam")

In [3]:
ROOT_DIR

NameError: name 'ROOT_DIR' is not defined

In [ ]:
# def parse(g):
#     for l in g:
#         yield eval(l)


# countU = defaultdict(lambda: 0)
# countP = defaultdict(lambda: 0)
# line = 0

# dataset_name = "Steam"
# json_file_path = os.path.join(ROOT_DIR, f"reviews_{dataset_name}.json.gz")

# with gzip.open(json_file_path, "r") as g:
#     line = 0
#     for l in parse(g):
#         line += 1
#         if line % 100000 == 0:
#             print(line)

#         product_id = int(l["product_id"])
#         username = l["username"]
#         time = datetime.strptime(l["date"], "%Y-%m-%d")
#         countU[username] += 1
#         countP[product_id] += 1

In [ ]:
# keys = set(countP.keys())
# ids = set(df['id'].unique())

In [ ]:
# df[df['id']==340]

In [ ]:
# keys - ids

In [ ]:
with open(os.path.join(ROOT_DIR, "steam_games.json")) as json_file:
    lines = json_file.read().splitlines()
records = [ast.literal_eval(line) for line in lines]

raw_df = pd.DataFrame(records)

In [ ]:
raw_df.info()

In [ ]:
df = raw_df[['id', 'sentiment', 'genres', 'specs', 'price', 'early_access']]

In [12]:
#fix id
df.dropna(subset = ['id'], inplace=True)
df['id'] = df['id'].astype(int)

itemmap = {}
with open(os.path.join(ROOT_DIR, "itemmap.pkl"), 'rb') as f:
    itemmap = pkl.load(f)

df['id'] = df['id'].apply(lambda x: itemmap[x] if x in  itemmap else None)
df.dropna(subset = ['id'], inplace=True)
df = df.drop_duplicates(subset=['id'])

df['id'] = df['id'].astype(int)
df = df.reset_index()
print(len(df))

13047
<ipython-input-12-9604afbdf72d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset = ['id'], inplace=True)
<ipython-input-12-9604afbdf72d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['id'].astype(int)
<ipython-input-12-9604afbdf72d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[

In [13]:
df

,index,id,sentiment,genres,specs,price,early_access
0,1,3061,Mostly Positive,"[Free to Play, Indie, RPG, Strategy]","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False
1,2,2019,Mostly Positive,"[Casual, Free to Play, Indie, Simulation, Sports]","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False
2,5,10834,Mixed,"[Action, Adventure, Simulation]","[Single-player, Steam Achievements]",3.99,False
3,27,6827,Very Positive,"[Action, Indie, Racing]","[Single-player, Multi-player, Steam Trading Ca...",9.99,False
4,28,3583,Overwhelmingly Positive,[Action],"[Single-player, Multi-player, Valve Anti-Cheat...",9.99,False
...,...,...,...,...,...,...,...
13042,32116,6392,Overwhelmingly Positive,[Action],"[Single-player, Multi-player, Steam Trading Ca...",14.99,False
13043,32117,2950,Very Positive,[Action],"[Single-player, Multi-player, Valve Anti-Cheat...",9.99,False
13044,32124,10483,4 user reviews,"[Adventure, Indie]","[Single-player, Steam Achievements]",0.99,False
13045,32129,9334,2 user reviews,"[Action, Adventure, Casual, Indie]","[Single-player, Steam Achievements, Steam Cloud]",1.99,False


In [14]:
#fix sentiment
sentiments = ['Unknown', 'Overwhelmingly Negative', "Very Negative", "Negative", "Mostly Negative", "Mixed", "Mostly Positive", "Positive" , "Very Positive" ,"Overwhelmingly Positive"]
sentiments_dict = {
    'Unknown': -1,
    'Overwhelmingly Negative': 0,
    'Very Negative': 1,
    'Negative': 2,
    'Mostly Negative': 3,
    'Mixed': 4,
    'Mostly Positive': 5,
    'Positive': 6,
    'Very Positive': 7,
    'Overwhelmingly Positive': 8,
}
df['sentiment'] = df['sentiment'].fillna('Unknown')
df['sentiment'] = df['sentiment'].apply(lambda x: x if x in sentiments else 'Unknown')
df['sentiment'] = df['sentiment'].apply(lambda x: sentiments_dict[x])
df['sentiment'] = df['sentiment'].astype(int)

In [15]:
#fix genres
genres = set()
for item in list(raw_df["genres"].dropna()):        
    genres = genres | set(item)
print(len(genres))

def is_in_genere(genre, x):
    try:
        return 1 if genre in x else 0
    except:
        return 0

for genre in genres:
    df[f'genre_{genre}'] = df['genres'].apply(lambda x: is_in_genere(genre=genre, x=x))

df.drop(columns=['genres'],inplace=True)

22


In [16]:
#fix specs
specs = set()
for item in list(raw_df["specs"].dropna()):        
    specs = specs | set(item)
print(len(specs))

def is_in_spec(spec, x):
    try:
        return 1 if spec in x else 0
    except:
        return 0

for spec in specs:
    df[f'spec_{spec}'] = df['specs'].apply(lambda x: is_in_spec(spec, x))

df.drop(columns=['specs'],inplace=True)

40


In [17]:
df

,index,id,sentiment,price,early_access,genre_Massively Multiplayer,genre_Sports,genre_Action,genre_Simulation,genre_Accounting,...,spec_Oculus Rift,spec_Steam Turn Notifications,spec_Partial Controller Support,spec_Includes level editor,spec_Seated,spec_Cross-Platform Multiplayer,spec_Steam Achievements,spec_Room-Scale,spec_In-App Purchases,spec_Steam Workshop
0,1,3061,5,Free To Play,False,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
1,2,2019,5,Free to Play,False,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,5,10834,4,3.99,False,0,0,1,1,0,...,0,0,0,0,0,0,1,0,0,0
3,27,6827,7,9.99,False,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,28,3583,8,9.99,False,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13042,32116,6392,8,14.99,False,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
13043,32117,2950,7,9.99,False,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
13044,32124,10483,-1,0.99,False,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
13045,32129,9334,-1,1.99,False,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [18]:
#fix price
# strange_prices = ['Free to Play', 'nan','Free','Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Third-party', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Starting at $499.00', 'Starting at $449.00', 'Free to Try', 'Free Movie', 'Free to Use']
free_prices = ['Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Free to Try', 'Free Movie', 'Free to Use', 'Free To Play	', "'Free To Play'", 'Free To Play']
free_prices = free_prices + [df.iloc[1]['price']]
df['price'] = df['price'].apply(lambda x: 0 if x in free_prices else x)
df['price'] = df['price'].apply(lambda x: 0 if x == 'Third-party' else x)
df['price'] = df['price'].apply(lambda x: 499.0 if x == 'Starting at $499.00' else x)
df['price'] = df['price'].apply(lambda x: 449.0 if x == 'Starting at $449.00' else x)
df['price'] = df['price'].fillna(-1)
# df['price'].iloc[0] = 0
# df.dropna(subset = ['price'], inplace=True)

# df['price']=df['price'].apply(lambda x:round(x,2))

In [19]:
df[df['price'] == "Free To Play"]

,index,id,sentiment,price,early_access,genre_Massively Multiplayer,genre_Sports,genre_Action,genre_Simulation,genre_Accounting,...,spec_Oculus Rift,spec_Steam Turn Notifications,spec_Partial Controller Support,spec_Includes level editor,spec_Seated,spec_Cross-Platform Multiplayer,spec_Steam Achievements,spec_Room-Scale,spec_In-App Purchases,spec_Steam Workshop


In [20]:
# fix early_access
df['early_access'] = df['early_access'].apply(lambda x: 1 if x else 0)

In [21]:
df[df.isnull().any(axis=1)]

,index,id,sentiment,price,early_access,genre_Massively Multiplayer,genre_Sports,genre_Action,genre_Simulation,genre_Accounting,...,spec_Oculus Rift,spec_Steam Turn Notifications,spec_Partial Controller Support,spec_Includes level editor,spec_Seated,spec_Cross-Platform Multiplayer,spec_Steam Achievements,spec_Room-Scale,spec_In-App Purchases,spec_Steam Workshop


In [22]:
df.to_csv(os.path.join(ROOT_DIR, "items_info.csv"), index=False)

In [23]:
df

,index,id,sentiment,price,early_access,genre_Massively Multiplayer,genre_Sports,genre_Action,genre_Simulation,genre_Accounting,...,spec_Oculus Rift,spec_Steam Turn Notifications,spec_Partial Controller Support,spec_Includes level editor,spec_Seated,spec_Cross-Platform Multiplayer,spec_Steam Achievements,spec_Room-Scale,spec_In-App Purchases,spec_Steam Workshop
0,1,3061,5,0.00,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
1,2,2019,5,0.00,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,5,10834,4,3.99,0,0,0,1,1,0,...,0,0,0,0,0,0,1,0,0,0
3,27,6827,7,9.99,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,28,3583,8,9.99,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13042,32116,6392,8,14.99,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
13043,32117,2950,7,9.99,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
13044,32124,10483,-1,0.99,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
13045,32129,9334,-1,1.99,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [38]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca_df = df.drop(columns=['id'])
pca = PCA(n_components=50)
scaler = StandardScaler()
pca_np = scaler.fit_transform(pca.fit_transform(pca_df))
pca_df = pd.DataFrame(pca_np)
pca_df['id'] = df['id']

columns = ['id']
for i in range(50):
    columns.append(i)
pca_df = pca_df[columns]

pca_df.to_csv(os.path.join(ROOT_DIR, "items_info_pca.csv"), index=False)

In [39]:
pca_df

,id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,3061,1.632144,-0.644972,-0.329914,-0.009933,-0.948314,2.002481,1.976254,-1.948210,0.208270,...,0.080944,-0.003862,-0.820910,0.315101,-0.116259,0.044188,0.011916,0.108588,-0.011955,0.105477
1,2019,1.632045,-0.645301,-0.302780,-1.198375,-0.822720,1.434227,1.115789,1.407753,0.143512,...,0.083933,-0.049179,-0.262532,0.325600,-0.131910,-0.012076,-0.018736,0.117220,0.009550,0.057176
2,10834,1.631747,-0.375631,0.091572,-0.397072,-0.004765,-1.056380,-0.524779,-0.555841,-0.450960,...,-0.039312,0.050003,0.136925,0.068571,-0.045924,0.012952,0.024151,-0.000507,0.014299,-0.032618
3,6827,1.629562,0.033241,-1.019878,-0.283925,-1.090579,-0.487366,0.437009,0.470111,-0.639924,...,-0.461612,-0.045931,-0.118752,-0.051114,0.099320,0.033045,0.012624,0.016627,0.002343,-0.066340
4,3583,1.629462,0.034706,-1.399206,-1.382571,-1.548570,-0.769859,-0.855055,0.131137,-0.536727,...,0.314680,-0.046243,0.177269,0.122827,-2.654367,-0.290698,-0.197154,-0.020494,0.162864,-1.077788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13042,6392,-1.557670,0.382623,-1.301733,-0.716717,-1.078528,-0.446285,-0.431903,-0.162771,-1.122818,...,-0.164987,0.033352,-0.060753,-0.039996,0.021779,0.005821,0.034540,-0.007128,-0.033608,-0.117395
13043,2950,-1.557770,0.043177,-0.940179,-1.267592,-1.468875,-0.697728,-0.803932,0.154455,-0.463746,...,0.304884,-0.029299,0.184556,0.090303,-2.681049,-0.281918,-0.161467,-0.027013,0.119155,-1.101736
13044,10483,-1.558465,-0.574767,2.069444,0.099359,1.018956,-0.531398,0.094170,-0.564127,0.463587,...,0.020173,0.025182,0.046932,0.014732,0.018386,0.027975,0.046928,-0.025646,-0.029352,-0.039118
13045,9334,-1.558962,-0.507289,2.069028,1.017051,0.776476,-0.646556,-0.111596,0.478762,-0.121486,...,-0.038190,-0.009450,0.082919,0.012080,-0.060144,0.032767,0.015577,-0.019862,-0.013027,-0.028627


In [36]:
from sklearn.cluster import FeatureAgglomeration
from sklearn.preprocessing import StandardScaler

fagg_df = df.drop(columns=['id'])
fagg = FeatureAgglomeration(n_clusters=50)
scaler = StandardScaler()
fagg_np = scaler.fit_transform(fagg.fit_transform(fagg_df))
fagg_df = pd.DataFrame(fagg_np)
fagg_df['id'] = df['id']

columns = ['id']
for i in range(50):
    columns.append(i)
fagg_df = fagg_df[columns]

fagg_df.to_csv(os.path.join(ROOT_DIR, "items_info_fagg.csv"), index=False)

In [37]:
fagg_df

,id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,3061,-0.043108,-0.059,-0.303436,5.727174,-0.216275,-0.183943,4.177802,-0.234671,-0.157548,...,-0.182154,-0.643787,-0.060128,-0.882046,-0.073445,-0.197749,-0.086547,-0.640858,-0.051115,-0.043816
1,2019,-0.043108,-0.059,-0.303436,5.727174,-0.216275,-0.183943,-0.239360,-0.234671,-0.157548,...,-0.182154,1.553308,-0.060128,-0.882046,-0.073445,-0.197749,-0.086547,-0.640858,-0.051115,-0.043816
2,10834,-0.043108,-0.059,-0.303436,-0.174606,-0.216275,-0.183943,-0.239360,-0.234671,-0.157548,...,-0.182154,-0.643787,-0.060128,1.133728,-0.073445,-0.197749,-0.086547,-0.370691,-0.051115,-0.043816
3,6827,-0.043108,-0.059,-0.303436,-0.174606,-0.216275,-0.183943,-0.239360,-0.234671,-0.157548,...,-0.182154,-0.643787,-0.060128,1.133728,-0.073445,5.056906,-0.086547,0.035576,-0.051115,-0.043816
4,3583,-0.043108,-0.059,-0.303436,-0.174606,-0.216275,-0.183943,-0.239360,-0.234671,-0.157548,...,-0.182154,-0.643787,-0.060128,1.133728,-0.073445,-0.197749,-0.086547,0.035576,-0.051115,-0.043816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13042,6392,-0.043108,-0.059,-0.303436,-0.174606,-0.216275,-0.183943,-0.239360,-0.234671,-0.157548,...,-0.182154,-0.643787,-0.060128,1.133728,-0.073445,-0.197749,-0.086547,0.374132,-0.051115,-0.043816
13043,2950,-0.043108,-0.059,-0.303436,-0.174606,-0.216275,-0.183943,-0.239360,-0.234671,-0.157548,...,-0.182154,-0.643787,-0.060128,1.133728,-0.073445,-0.197749,-0.086547,0.035576,-0.051115,-0.043816
13044,10483,-0.043108,-0.059,-0.303436,-0.174606,-0.216275,-0.183943,-0.239360,-0.234671,-0.157548,...,-0.182154,-0.643787,-0.060128,-0.882046,-0.073445,-0.197749,-0.086547,-0.573824,-0.051115,-0.043816
13045,9334,-0.043108,-0.059,-0.303436,-0.174606,-0.216275,-0.183943,-0.239360,-0.234671,-0.157548,...,-0.182154,1.553308,-0.060128,1.133728,-0.073445,-0.197749,-0.086547,-0.506113,-0.051115,-0.043816
